In [1]:
import pandas as pd
import torch
from torch import device, cuda, autocast
from torch.cuda.amp import GradScaler
from torch.nn import BCEWithLogitsLoss
from tqdm import tqdm
import wandb

import flyvision
from flyvision_ans import DECODING_CELLS
from flyvision.utils.activity_utils import LayerActivity
from from_retina_to_connectome_funcs import from_retina_to_model, get_decision_making_neurons, get_cell_type_indices
from graph_models import GNNModel
from from_video_to_training_batched_funcs import get_files_from_directory, select_random_videos, paths_to_labels, \
    load_custom_sequences

device_type = "cuda" if cuda.is_available() else "cpu"
DEVICE = device(device_type)
torch.manual_seed(42)
batch_size = 2
last_good_frame = 2

TRAINING_DATA_DIR = "easy_videos"
TESTING_DATA_DIR = "easyval_videos"

debugging = True
wandb_ = True

NUM_CONNECTOME_PASSES=10

/home/eudald/Desktop/doctorat/connectome/venv/lib/python3.10/site-packages/torch/__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:451.)
  _C._set_default_tensor_type(t)


In [2]:
# init stuff
extent, kernel_size = 15, 13
decision_making_vector = get_decision_making_neurons()
receptors = flyvision.rendering.BoxEye(extent=extent, kernel_size=kernel_size)
network_view = flyvision.NetworkView(flyvision.results_dir / "opticflow/000/0000")
network = network_view.init_network(chkpt="best_chkpt")
classification = pd.read_csv("adult_data/classification_clean.csv")
root_id_to_index = pd.read_csv("adult_data/root_id_to_index.csv")

all_videos = get_files_from_directory(TRAINING_DATA_DIR)
all_validation_videos = get_files_from_directory(TESTING_DATA_DIR)
dt = 1 / 100 # some random parameter from flyvision

cell_type_indices = get_cell_type_indices(classification, root_id_to_index, DECODING_CELLS)


In [3]:
model = GNNModel(
    num_node_features=1, 
    decision_making_vector=decision_making_vector,
    num_passes=NUM_CONNECTOME_PASSES,
    cell_type_indices=cell_type_indices,
    batch_size=batch_size
).to(DEVICE)
lr = .01
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scaler = GradScaler()

# Initialize the loss function
criterion = BCEWithLogitsLoss()

In [4]:
if wandb_:
    wandb.init(project="adult_connectome", config={"learning_rate": lr, "batch_size": batch_size})

already_selected = []
for i in tqdm(range(len(all_videos) // batch_size)):
    batch_files, already_selected = select_random_videos(all_videos, batch_size, already_selected)
    labels = paths_to_labels(batch_files)  # Assuming this function converts paths to labels
    batch_sequences = load_custom_sequences(batch_files)
    
    # Assuming receptors is a function that processes your sequences
    rendered_sequences = receptors(batch_sequences)
    
    layer_activations = []
    for rendered_sequence in rendered_sequences:
        simulation = network.simulate(rendered_sequence[None], dt)
        layer_activations.append(LayerActivity(simulation, network.connectome, keepref=True))
        
    del rendered_sequences, simulation
    torch.cuda.empty_cache()
    
    # Assuming from_retina_to_model prepares your data for the model, including labels
    inputs, labels = from_retina_to_model(
        layer_activations, labels, DECODING_CELLS, last_good_frame, classification, root_id_to_index
    )
    torch.cuda.empty_cache()
    
    model.train()
    if wandb_:
        wandb.watch(model, criterion, log="all", log_freq=10)
    # Update the code to process the entire batch at once
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE).unsqueeze(-1).float() 
    optimizer.zero_grad()
    
    with autocast(device_type):
        out = model(inputs)
        loss = criterion(out, labels)
    
    if wandb_:
        wandb.log({"loss": loss.item()})
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()
    optimizer.zero_grad()
    
    if debugging and i == 99:
        break

wandb: Currently logged in as: eudald. Use `wandb login --relogin` to force relogin


  1%|          | 99/8400 [22:06<30:53:59, 13.40s/it]


In [ ]:
already_selected_validation = []
# Assuming batch_size is defined
for _ in tqdm(range(len(all_validation_videos) // batch_size)):
    batch_files, already_selected_validation = select_random_videos(all_validation_videos, batch_size, already_selected_validation)

    labels = paths_to_labels(batch_files)  # Convert paths to labels
    batch_sequences = load_custom_sequences(batch_files)  # Load and preprocess the video sequences
    
    # Assuming receptors is a function that processes your sequences
    rendered_sequences = receptors(batch_sequences)
    
    layer_activations = []
    for rendered_sequence in rendered_sequences:
        simulation = network.simulate(rendered_sequence[None], dt)
        layer_activations.append(LayerActivity(simulation, network.connectome, keepref=True))
        
    del rendered_sequences, simulation
    torch.cuda.empty_cache()
    
    # Preparing the data for the model, similar to training
    inputs, labels = from_retina_to_model(layer_activations, labels, DECODING_CELLS, last_good_frame, classification, root_id_to_index)
    torch.cuda.empty_cache()

    model.eval()  # Set the model to evaluation mode
    val_loss = []
    with torch.no_grad():  # Disable gradient computation
        inputs = inputs.to(DEVICE)
        labels = labels.to(DEVICE).unsqueeze(-1).float()
        
        with autocast(device_type):
            predictions = model(inputs)
            # Assuming your criterion and evaluation metrics are defined similarly to training
            loss = criterion(predictions, labels)
            val_loss.append(loss.item())
            # Calculate other metrics if necessary, e.g., accuracy
            
            # Log validation metrics to WandB
            wandb.log({"validation_loss": loss.item()})
            # Log other metrics similarly

When cuda breaks:
sudo rmmod nvidia_uvm
sudo modprobe nvidia_uvm